#Corpus Generator
Generates a corpus for training using the titles of CamHarvestCollection

In [1]:
import pymongo
from pymongo import MongoClient
import nltk
from nltk.corpus import stopwords
import string
import codecs
import gensim 
from gensim import corpora, models
punct_filter = dict((ord(char), u' ') for char in '"#$%&\'()*+,./-:;<=>?@[\\]^_`{|}')   
stop = stopwords.words('english')
mongo_url = 'mongodb://localhost:6666/'
db = 'Cherry'
coll_in = 'Cranberry'
client = MongoClient(mongo_url)
ch = client[db][coll_in]
#corpusfile = 'corpus2.txt' 


In [2]:
def raw_corpus_generator():
    ind = 0 
    with codecs.open('raw_corpus.txt','a',encoding='utf8') as f:
        for rec in ch.find({'crossref_doi':True}):
            lt = rec['title'].lower()
            slt = lt.strip()
            tslt = slt.translate(punct_filter)
            export = tslt+u'\n'
            f.write(export)
            ind+=1
            if ind%100000==0:
                print(ind)

def sanitise(title):
    lt = title.lower()
    slt = lt.strip()
    tslt = slt.translate(punct_filter)
    stop_filtered = [i for i in tslt.split() if i not in stop]
    export = u' '.join(stop_filtered)+u'\n'
    return export

def create_stopword_filtered_corpus():
    ind = 0 
    with codecs.open('second_corpus.txt','a',encoding='utf8') as f:
        for rec in ch.find({'crossref_doi':True}):
            f.write(sanitise(rec['title']))
            ind+=1
            if ind%10000==0:
                print(ind)

In [3]:
class GensimCorpus(object):
    def __init__(self,corpus_text_file,diction):
        self.corpus_text_file = corpus_text_file
        self.dictionary = diction
        
    def __iter__(self):
        for line in open(self.corpus_text_file):
            yield self.dictionary.doc2bow(line.split())

def dictionary_generator(corpus_file):
    dictionary = corpora.Dictionary(line.split() for line in open(corpus_file))
    return dictionary

In [4]:
#dictionary = dictionary_generator('second_corpus.txt')
dictionary = corpora.Dictionary.load('second_dictionary')
corp = GensimCorpus('second_corpus.txt',dictionary)
tfidf = models.TfidfModel(corp)
tfidf_corp = tfidf[corp]
#tfidf.save('second_tfidf')

In [5]:
def tfidf_filtered_corpus_generator(threshold):
    corpus_filename = 'tfidf_filtered_'+str(threshold).strip('.')+'.txt'
    ind=0
    with codecs.open(corpus_filename,'a',encoding='utf8') as f:
        for doc in tfidf_corp:
            if ind%500000 == 0:
                print(ind)
            f.write(' '.join([dictionary[i] for i,j in doc if j>=threshold]))
            f.write('\n')
            ind+=1

In [6]:
samples = [0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
for samp in samples:
    tfidf_filtered_corpus_generator(samp)
    print('corpus generated')

0
500000
1000000
corpus generated
0
500000
1000000
corpus generated
0
500000
1000000
corpus generated
0
500000
1000000
corpus generated
0
500000
1000000
corpus generated
0
500000
1000000
corpus generated
0
500000
1000000
corpus generated
0
500000
1000000
corpus generated
0
500000
1000000
corpus generated
